In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import json
import re
from time import sleep

import random 
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
%run C:\Users\a.nematov\Desktop\Парсеры\useful_functions.ipynb

In [3]:
session = rq.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

meta = {
    'leningradskaya_oblast' : 636370,
    'sankt-peterburg' : 653240
}

districts = {
    'Адмиралтейский' : 762,
    'Василеостровский' : 763,
    'Выборгский' : 764,
    'Калининский' : 765,
    'Кировский' : 766,
    'Колпинский' : 767,
    'Красногвардейский' : 768,
    'Красносельский' : 769,
    'Московский' : 772,
    'Невский' : 773,
    'Петроградский' : 774,
    'Приморский' : 776,
    'Фрунзенский' : 778,
    'Центральный' : 779
}

highways = {
    'Александровское шоссе' : 33,
    "Аннинское шоссе" : 34,
    "Белоостровское шоссе": 35,
    "Волхонское шоссе": 36,
    "Выборгское шоссе": 37,
    "Гатчинское шоссе": 38, 
    "Горское шоссе": 39, 
    "Гостилицкое шоссе": 126,
    "Дорога жизни": 130,
    "Зеленогорское шоссе": 40,
    "Киевское шоссе": 41,
    "Красносельское шоссе": 42,
    "Левашовское шоссе": 43,
    "Ленинградское шоссе": 127,
    "Московское шоссе": 44,
    "Мурманское шоссе": 51,
    "Петрозаводское шоссе": 45,
    "Приморское шоссе": 46,
    "Приозерское шоссе": 47,
    "Пулковское шоссе": 48, 
    "Ропшинское шоссе": 49,
    "Рябовское шоссе": 50,
    "Таллинское шоссе": 122,
    "Трасса Вологда — Новая Ладога": 128,
    "Трасса Лодейное поле — Вытегра": 125,
    "Трасса Лодейное Поле — Тихвин": 129,
    "Трасса \"Скандинавия\"": 124,
    "Трасса \"Сортавала\"": 123
}


In [4]:
def parse_header(raw_header: str):
    header = dict()
    for line in raw_header.split("\n"):
        if line.startswith(":"):
            a, b = line[1:].split(":", 1)
            a = f":{a}"
        else:
            a, b = line.split(":",1)
        header[a.strip()] = b.strip()
    return header

In [5]:
def decod_response(content):    
    dctx = zstd.ZstdDecompressor()
    stream_reader = dctx.stream_reader(BytesIO(content))
    decompressed_data = stream_reader.read()
    decod = decompressed_data.decode('utf-8')
    
    return decod


def get_json(page, limit, location, district):

    url = f'https://www.avito.ru/{location}/kommercheskaya_nedvizhimost/prodam-ASgBAgICAUSwCNJW?map=e30%3D'

    if district:
        url = f'https://www.avito.ru/{location}/kommercheskaya_nedvizhimost/prodam-ASgBAgICAUSwCNJW?district={district}&map=e30%3D'
    
    headers = f"""Accept:application/json
        Accept-Encoding:gzip, deflate, br, zstd
        Accept-Language:ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
        Cookie:srv_id=_in8v3cPGv439F1t.pv8DEvFEE2qCTRTC8mocn506JPVjab2MGLRnCDcC0BKVBqY7wqbkGy1pHnbJPoo=.sKLm4xKVnEfbYMzYsHZIYcXfPt2soPgWhVLiJ35htyE=.web; u=2y7rcusd.psxc9a.1lc4q5ogi2900; tmr_lvid=2b65124eda4cc7012869344c58428214; tmr_lvidTS=1702044174392; _ym_uid=1702044175999009250; adrcid=AUMnqFXmD8Y1OXl_jYF1hSw; uxs_uid=7c247960-95d2-11ee-9f59-ada3bc1b2bbc; __upin=d9SIlydAFpwg49C6OdieBw; __zzatw-avito=MDA0dBA=Fz2+aQ==; __zzatw-avito=MDA0dBA=Fz2+aQ==; adrcid=AUMnqFXmD8Y1OXl_jYF1hSw; __ddg1_=Dhkdi3kP0rEPVfHgn7eA; adrcid=AUMnqFXmD8Y1OXl_jYF1hSw; _buzz_fpc=JTdCJTIydmFsdWUlMjIlM0ElN0IlMjJ1ZnAlMjIlM0ElMjI2Nzc3OTg0MDdjYzhiMWEwYWFiZTMyMzg0MjRmNzJjOSUyMiUyQyUyMmJyb3dzZXJWZXJzaW9uJTIyJTNBJTIyMTI1LjAlMjIlMkMlMjJ0c0NyZWF0ZWQlMjIlM0ExNzE3NTg1NjQyMDA3JTdEJTJDJTIycGF0aCUyMiUzQSUyMiUyRiUyMiUyQyUyMmRvbWFpbiUyMiUzQSUyMi53d3cuYXZpdG8ucnUlMjIlMkMlMjJleHBpcmVzJTIyJTNBJTIyVGh1JTJDJTIwMDUlMjBKdW4lMjAyMDI1JTIwMTElM0EwNyUzQTIyJTIwR01UJTIyJTJDJTIyU2FtZVNpdGUlMjIlM0ElMjJMYXglMjIlN0Q=; _buzz_aidata=JTdCJTIydmFsdWUlMjIlM0ElN0IlMjJ1ZnAlMjIlM0ElMjJkOVNJbHlkQUZwd2c0OUM2T2RpZUJ3JTIyJTJDJTIyYnJvd3NlclZlcnNpb24lMjIlM0ElMjIxMjUuMCUyMiUyQyUyMnRzQ3JlYXRlZCUyMiUzQTE3MTc1ODU2NDIwNjYlN0QlMkMlMjJwYXRoJTIyJTNBJTIyJTJGJTIyJTJDJTIyZG9tYWluJTIyJTNBJTIyLnd3dy5hdml0by5ydSUyMiUyQyUyMmV4cGlyZXMlMjIlM0ElMjJUaHUlMkMlMjAwNSUyMEp1biUyMDIwMjUlMjAxMSUzQTA3JTNBMjIlMjBHTVQlMjIlMkMlMjJTYW1lU2l0ZSUyMiUzQSUyMkxheCUyMiU3RA==; _ga_DQYB5Z23VK=GS1.2.1717675259.3.1.1717675308.0.0.0; _ym_d=1718088991; yandex_monthly_cookie=true; cfidsw-avito=xJJALdKyIsMNh5b4m4TYkgkSB0DWQTVSygGIsuLfzJ8zyJB/mCo6jISgclACMjrsN/EQtfQb5dPdVeXLgP5OILlKeAxWovemkz4h1bUr5Ne3KdItNpCh046oOXe98xyzxkpmty37NzhJD21XsvGyVew4JkmtumwEZUvf; cfidsw-avito=xJJALdKyIsMNh5b4m4TYkgkSB0DWQTVSygGIsuLfzJ8zyJB/mCo6jISgclACMjrsN/EQtfQb5dPdVeXLgP5OILlKeAxWovemkz4h1bUr5Ne3KdItNpCh046oOXe98xyzxkpmty37NzhJD21XsvGyVew4JkmtumwEZUvf; sessid=37acdad71abb5a5fcd0e3f9245d3dc3c.1718089401; cfidsw-avito=CXVYIvq78UZotgxu5443/ZLpXeW7SwwwBKkqM2owhAUFPtd2Oa0FcK2GOwSHF5y4SZt30+uw6YjP62j9NnZEMirdH7+SD1b8WvuvoByBeqdf9dExYa09rJQkbp+eTU4BHEOoThmhTQ8yucAZ+E6+FqB9ujfRFEfVkDpP; _ga=GA1.1.1628045832.1702044174; SEARCH_HISTORY_IDS=1; buyer_popup_location=0; _gcl_au=1.1.2115663959.1718260570; gMltIuegZN2COuSe=EOFGWsm50bhh17prLqaIgdir1V0kgrvN; adrdel=1718779186260; adrdel=1718779186260; _ym_isad=2; _ga_ZJDLBTV49B=GS1.1.1718779491.16.0.1718779500.0.0.0; _ga_WW6Q1STJ8M=GS1.1.1718779491.19.0.1718779500.0.0.0; v=1718781547; _ym_visorc=b; domain_sid=wCWfUQosfT7qCIjcI31wf%3A1718783933850; f=5.0c4f4b6d233fb90636b4dd61b04726f147e1eada7172e06c47e1eada7172e06c47e1eada7172e06c47e1eada7172e06cb59320d6eb6303c1b59320d6eb6303c1b59320d6eb6303c147e1eada7172e06c8a38e2c5b3e08b898a38e2c5b3e08b890df103df0c26013a7b0d53c7afc06d0b2ebf3cb6fd35a0ac0df103df0c26013a8b1472fe2f9ba6b9ad42d01242e34c7968e2978c700f15b6831064c92d93c3903815369ae2d1a81d4e0d8a280d6b65f00df103df0c26013aba0ac8037e2b74f9268a7bf63aa148d22ebf3cb6fd35a0ac8b1472fe2f9ba6b97b0d53c7afc06d0b71e7cb57bbcb8e0f03c77801b122405c03c77801b122405c2da10fb74cac1eab2ebf3cb6fd35a0ac20f3d16ad0b1c546b892c6c84ad16848a9b4102d42ade879dcb5a55b9498f642baf80da35caa52287658d123ba269e0348fe2c34fc387c28b1d542c59d0bd3144525907271a6a0eb69a2241f7870d4d8f4857885524eb1f691e52da22a560f550df103df0c26013a0df103df0c26013aaaa2b79c1ae92595846346fd022b4d1da98cd3726d66dcc53de19da9ed218fe2c772035eab81f5e123f5e56da7ec04f4a1a4201a28a6ec9b059080ed9becc4cd; ft="pBDTu6J8CnoSp8sSEd1uAz7+JsNiPBojA04MMJTxjB8egHm7MzZG9Lhgjtl5CADw9uCdUKaojgVMKImiBYvoRlVQx4Nx8rtYqg6QGZQ10jcJ2dNimZzMnol79EvxtuEkmZbClL87yFd5BOyT6bLxHUYvUs2HEnGGPkP0cUjNfMaGlxhEX/prP5JpglrgVyKR"; isLegalPerson=0; buyer_laas_location=636370; cartCounter=0; dfp_group=94; tmr_detect=0%7C1718785264296; buyer_location_id=653240; luri=sankt-peterburg; sx=H4sIAAAAAAAC%2F1yWW47zvK5E55LnfhB1IaV%2FNhQlOoljx45vsTd67gdpoPOd9AQWCuWlov93MhA1CmSjAYVtFNESndhQSTUgnv7732k9%2FXeCo30M58vd26fl%2FfJEO17aNKzbNO1jH09fp3r6DwhiJJNs%2FP46GTbkixEjmlFZnGQrEkLWUCWR%2FyVfH7xmez5gcYN9tHPw8ej21E19P81SP8jOp%2B%2BvE0QIFotQDMZYkhKVQyC1qBAylF%2Fy6FrtoR0fDFyb6zbLfegV9RryWW%2F9n8w%2F5CqSHIqQupzAFImpECG5HFkQfsnzY5183z7PXOuIdZ%2F3SyqhM%2F5yu91n%2FSBDsN9fJ2vEKCRmyFyct1wh2aqmuJp8rO%2Ber22ye50Z9Lz3174ILwC39gIz6eo%2B24gUXuRMYL2Cj1ZiVY%2BgFMFXdaWC%2FGujWaFoJ5tt1m0KE861HL008bwdj7Z2nz07%2BP46OUOAJoKpNid1mrNJmCKIQHRq4i%2F5st0IfMYjxvuxedc%2FxuVyrX64W4gpfLZhzIscAudSFZjQ%2BcrVhVCc%2BsylUME32ZUnXLpe99DN%2BkjL%2FRlGe4u3vkrs4PMLQnxZ5wRclIiojKXWwEUUyfloUhVf3tZlHfbLPJQaqJm7s3YNNEfCDZbu2P%2BQY3TfXydfgQybXFLxhkRihmJUjCeTKQX5JZd7PZ73o2Uz9ClFm6OdDyoCAQ095v9PBgwv60JEZ6kmlaKQvTUmKfqsUKIHjPmXHNf5wWGL3bWmIx%2FYbv58gTl3DTyneftDfrWBJjOa4LE6KyoRwCNQNWgiQUj2l2y6oyNzyH0ft%2FkGw1meZa%2Bt9niomT%2FcAICXz8iVnS21kmUkL058rikH4egE5Z3Z5RkaviwRVn6e73OUzdStzW2obsnL5xs09CJXrOiIqlUpMUUsSGp8tuJdBg6%2F5N1TxlHPbm0PeUx3213Ptkujm0pDJX1m%2FvGZfIaQNZkcapFKmclJZsXElI1%2Fv%2B50358RR7mmlPYhmKH0Yz%2BiA2tGb%2BY%2FPr%2FaoBqLt6lYNrYUYUwejVSo4F%2Fp39ZdLMylkyn2nPfdTsTcHsnEcLXdAO0H2aD%2F%2FjpFGyOiyWgMUAmmqNhgHLLGiiW%2FrbvPadqXPDyeMNsR94cOrqnFNcvcbs%2F8pw37Q2YJDjlHSpyKwVASCdVoNNeS63vr6tM9Bn9stLdj0YXHsE1XGueRt%2BVx%2FLHuh4yJocTsiitUk3hmEc9ZfbWx6Nu6kaFPUtPzGALoho%2FzvKThGG57u8X817qfNmKoOcScUyCHzKSkWIOQLZVNdb%2Fk1cEx8NSvD1P3KZWhlu65rmuTl3692w8ywcuNyLa8ms4aiUPJ6NH4bDin4G36t3VbP4mOF7muyv4Byak5%2BHy9jWV1ksY%2Fy%2F%2BTWWJwldkEsL4aCtnEbIDIe%2BYE78wjdcZMtDaXeru6%2FpkiN%2FFZqVmYrk36XKSf3RDwCTVQsPX1IYFF1RIKOiPoxbyt6zWZx05Lf2k2CWtY8LEtZVwAp9odn9bRD5mBMmokJMvWMjJKYIXXPQyhvG%2FKZFXg8sDU1UV8N0xh1ss5TeCGzOr%2FvO7XtRJRR%2BhKYFJTgq0UPVewPnhbjX2TO%2Bf3S2stNJaaisj2NXMtCc%2FYQP6T%2BbUbxZLVDGRSqQ7ROrDF%2BpSjaDRO3m8wdjlvgRtjhmXal9ta4uo8sHFUOvq8g4Cva1WY0VnGrIQeNVTF6CUbKSUH889nG7tGQ7wFwzTz4hHd0V1oTnMT%2Bj9%2FBQZf%2B1xD9bUEIGUKXqoEjjWLxRg5BKO%2F5Nbq9Xbe7aXezyHlbWvdbT1Khc5vsaVPN37ImhQkCTjna86UwCWpkpNoMOzkH7m9z3mZBRfCFAtfboNbhrZuHe7tMXxmTi83NBsuyiWkDCn4QhwAs1GKSZJYfu%2FzbeBrOd9Te7WS%2B9B0gI8mDfGAcMY%2FPYf4%2Ff1%2FAQAA%2F%2F9rLuDUjQoAAA%3D%3D; abp=0; pageviewCount=182; _ga_M29JC28873=GS1.1.1718779185.73.1.1718785312.59.0.0
        Priority:u=1, i
        Referer:{url}
        Sec-Ch-Ua:"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"
        Sec-Ch-Ua-Mobile:?0
        Sec-Ch-Ua-Platform:"Windows"
        Sec-Fetch-Dest:empty
        Sec-Fetch-Mode:cors
        Sec-Fetch-Site:same-origin
        User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36
        X-Requested-With:XMLHttpRequest
        X-Source:client-browser"""
    
    headers = parse_header(headers)
    
    source = f'https://www.avito.ru/js/1/map/items?categoryId=42&locationId={meta[location]}&correctorMode=0&page={page}&map=e30%3D&params%5B536%5D=5545&verticalCategoryId=1&rootCategoryId=4&localPriority=0&disabledFilters%5Bids%5D%5B0%5D=byTitle&disabledFilters%5Bslugs%5D%5B0%5D=bt&viewPort%5Bwidth%5D=672&viewPort%5Bheight%5D=765&limit={limit}&countAndItemsOnly=1'

    if district:
        source = f'https://www.avito.ru/js/1/map/items?categoryId=42&locationId={meta[location]}&districtId%5B0%5D={district}&correctorMode=0&page={page}&map=e30%3D&params%5B536%5D=5545&verticalCategoryId=1&rootCategoryId=4&localPriority=0&disabledFilters%5Bids%5D%5B0%5D=byTitle&disabledFilters%5Bslugs%5D%5B0%5D=bt&viewPort%5Bwidth%5D=672&viewPort%5Bheight%5D=679&limit={limit}&countAndItemsOnly=1'

    for req in range(5):
        if req != 0:
            print('Попытка {}'.format(req))
        try:
            resp = session.get(
                f"{source}",
                headers=headers
            )
    
            if 'status' in resp:
                print(f'Ошибка подключение:\nПодключаюсь через прокси')
                resp = session.get(
                    f"{source}",
                    headers=headers,
                    proxies = proxies
                )

            if 'error' not in resp:
                break
        except Exception as e:
            print('ERROR')
            print(e)
            resp = session.get(
                f"{source}",
                headers=headers,
                proxies = proxies
            )
    if resp.headers['content-encoding'] == 'zstd':
        response = decod_response(resp.content)
    else:
        response = resp.content
    return json.loads(response)

def get_data(page, limit, location, district):
    request = get_json(page, limit, location, district)
    
    if not request:
        return False
        
    try:
        data = request['items']
    except:
        print(json)
        return False
        
    local = [
        {
            "url": f'https://www.avito.ru{obj["urlPath"]}',
            'id' : obj['id'],
            'name' : obj['title'],
            'type' : obj['category']['name'],
            'seller' : '' if not obj['iva']['UserInfoStep'] or 'UserInfoStep' not in obj['iva']\
                        else obj['iva']['UserInfoStep'][0]['payload']['profile']['title'] ,
            'square' : '' if not re.search('([\d|.|,]+)м²', obj['title'].replace('\xa0', '') )
                       else re.search('([\d|.|,]+)м²', obj['title'].replace('\xa0', '') )[0],
            'price' : re.search('[\d]+' ,str(obj['priceDetailed']['value']))[0],
            'price_м²' : '' if not obj['normalizedPrice'] else re.search('[\d]+', obj['normalizedPrice'].replace('\xa0', ''))[0], 
            'timestamp_published' : obj['sortTimeStamp'],
            'address' : obj['geo']['formattedAddress'],
            'picture' : '' if not obj['images'] else obj['images'][0][list(obj['images'][0].keys())[0]],
            'metro' : '' if not obj['geo']['geoReferences'] else obj['geo']['geoReferences'][0]['content'],
            'time_to_metro' : '' if not obj['geo']['geoReferences'] or 'afterWithIcon' not in obj['geo']['geoReferences'][0]\
                              else obj['geo']['geoReferences'][0]['afterWithIcon']['text'],
            'distance_to_metro' : '' if not obj['geo']['geoReferences'] or not 'after' in obj['geo']['geoReferences'][0] \
                                  else obj['geo']['geoReferences'][0]['after'],
            'date_published' : obj['iva']['DateInfoStep'][0]['payload']['absolute'],
            'description' : obj['description'],
            'location': obj['location']['name'],
            'district': district,
            'latitude' : obj['coords']['lat'],
            'longitude' : obj['coords']['lng'],
            'type_deal': 'sale'
        }
        for obj in data
    ]
    
    return local

# Лен область

In [6]:
alldata = []

In [7]:
page = 1
limit = 10
local = get_data(page, limit, list(meta.keys())[0], None  )

In [8]:
limit = 50
page = 1

while True:
    local = get_data(page, limit, list(meta.keys())[0], None  )

    if not local:
        break
    
    print(f'На странице {page} собраны {len(local)} объявлений')
    alldata += local
    page += 1
    sleep(random.randint(1, 5))


На странице 1 собраны 50 объявлений
На странице 2 собраны 50 объявлений
На странице 3 собраны 50 объявлений
На странице 4 собраны 50 объявлений
На странице 5 собраны 50 объявлений
На странице 6 собраны 50 объявлений
На странице 7 собраны 50 объявлений
На странице 8 собраны 50 объявлений
На странице 9 собраны 50 объявлений
На странице 10 собраны 49 объявлений
На странице 11 собраны 50 объявлений
На странице 12 собраны 50 объявлений
На странице 13 собраны 50 объявлений
На странице 14 собраны 49 объявлений
На странице 15 собраны 49 объявлений
На странице 16 собраны 50 объявлений
На странице 17 собраны 50 объявлений
На странице 18 собраны 50 объявлений
На странице 19 собраны 50 объявлений
На странице 20 собраны 50 объявлений
На странице 21 собраны 50 объявлений
На странице 22 собраны 50 объявлений
На странице 23 собраны 50 объявлений
На странице 24 собраны 46 объявлений


# Санкт Петербург

In [9]:
limit = 50

for key, value in list(districts.items()):
    print(key)
    page = 1
    while True:
        local = get_data(page, limit, list(meta.keys())[1], value)
        
        if not local:
            break
        
        print(f'На странице {page} собраны {len(local)} объявлений')
        alldata += local
        page += 1
        sleep(random.randint(1, 5))


Адмиралтейский
На странице 1 собраны 49 объявлений
На странице 2 собраны 50 объявлений
На странице 3 собраны 50 объявлений
На странице 4 собраны 49 объявлений
На странице 5 собраны 50 объявлений
На странице 6 собраны 49 объявлений
На странице 7 собраны 50 объявлений
На странице 8 собраны 50 объявлений
На странице 9 собраны 28 объявлений
Василеостровский
На странице 1 собраны 50 объявлений
На странице 2 собраны 48 объявлений
На странице 3 собраны 50 объявлений
На странице 4 собраны 50 объявлений
На странице 5 собраны 50 объявлений
На странице 6 собраны 50 объявлений
На странице 7 собраны 46 объявлений
На странице 8 собраны 50 объявлений
На странице 9 собраны 40 объявлений
Выборгский
На странице 1 собраны 50 объявлений
На странице 2 собраны 50 объявлений
На странице 3 собраны 48 объявлений
На странице 4 собраны 50 объявлений
На странице 5 собраны 50 объявлений
На странице 6 собраны 50 объявлений
На странице 7 собраны 49 объявлений
На странице 8 собраны 50 объявлений
На странице 9 собраны

In [10]:
new_data = pd.DataFrame(alldata).drop_duplicates().reset_index(drop = True)
new_data['date_supple'] = new_data['timestamp_published'].apply(lambda x: datetime.fromtimestamp(int(str(x)[:-3])).strftime('%d.%m.%Y'))
new_data.drop_duplicates()

,url,id,name,type,seller,square,price,price_м²,timestamp_published,address,...,time_to_metro,distance_to_metro,date_published,description,location,district,latitude,longitude,type_deal,date_supple
0,https://www.avito.ru/volosovo/kommercheskaya_n...,4476393755,"Продажа производственного комплекса, ло",Коммерческая недвижимость,FOG Group,,420000000,6714,1732605059000,Ленинградская обл.,...,,,26 ноября 10:10,Арт. 74875861.\n\nПредлагается к продаже произ...,Волосово,NaN,59.337017,29.608975,sale,26.11.2024
1,https://www.avito.ru/volosovo/kommercheskaya_n...,2548487335,"Склад, 20 687 м² на участке 7,74 Га",Коммерческая недвижимость,NEVA PROPERTY,,310000000,14985,1731924653000,"Ленинградская область, Волосово, усадьба ВИЗ, 14",...,,,18 ноября 13:10,Продажа складского комплекса общей площадью 20...,Волосово,NaN,59.442357672988905,29.508250777954107,sale,18.11.2024
2,https://www.avito.ru/volosovo/kommercheskaya_n...,3897163631,Готовый действующий бизнес,Коммерческая недвижимость,МИР КВАРТИР PREMIUM,,430000000,107500,1732118872000,"Ленинградская обл., Волосово, ул. Ветеранов, 68А",...,,,20 ноября 19:07,Арт. 61683870 Продается готовый действующий би...,Волосово,NaN,59.444759,29.509424,sale,20.11.2024
3,https://www.avito.ru/volosovo/kommercheskaya_n...,3482990530,"Торговая площадь, 67 м²",Коммерческая недвижимость,,,20850000,311194,1732090878000,"Ленинградская область, Волосово, проспект Винг...",...,,,20 ноября 11:21,Предлагаем рассмотреть покупку коммерческого п...,Волосово,NaN,59.44229245099284,29.478185974932998,sale,20.11.2024
4,https://www.avito.ru/volosovo/kommercheskaya_n...,4714342358,"Свободного назначения, 730 м²",Коммерческая недвижимость,Абсолют Партнёр,,20000000,27397,1732430694000,"Ленинградская область, Волосово, проспект Винг...",...,,,24 ноября 09:44,Предлагаем вниманию инвесторов и предпринимате...,Волосово,NaN,59.442718,29.478512,sale,24.11.2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5292,https://www.avito.ru/sankt-peterburg/kommerche...,3033440481,Два особняка на набережной Невы,Коммерческая недвижимость,МИР КВАРТИР PREMIUM,,520000000,303738,1731770043000,"Санкт-Петербург, Синопская наб., 66-68-70Б",...,16–20 мин.,"1,5 км",16 ноября 18:14,"Арт. 45478836 Уважаемые покупатели, вашему вни...",Санкт-Петербург,779.0,59.936796,30.394516,sale,16.11.2024
5293,https://www.avito.ru/sankt-peterburg/kommerche...,4180617317,Комплекс Зданий по адресу Синопская наб. 68-70,Коммерческая недвижимость,Estate Market Agency,,560000000,290155,1732527592000,"Санкт-Петербург, Синопская наб., 66-68-70Б",...,16–20 мин.,"1,5 км",25 ноября 12:39,Предлагаем в продажу комплекс зданий с земельн...,Санкт-Петербург,779.0,59.936796,30.394516,sale,25.11.2024
5294,https://www.avito.ru/sankt-peterburg/kommerche...,3270162827,"Свободного назначения, 214.6 м²",Коммерческая недвижимость,HOME INVEST,,160500000,747903,1732288183000,"Санкт-Петербург, Орловская улица, 1к2",...,от 31 мин.,2 км,22 ноября 18:09,Home Invest — ваш партнер в поиске лучшего жил...,Санкт-Петербург,779.0,59.952132,30.391659,sale,22.11.2024
5295,https://www.avito.ru/sankt-peterburg/kommerche...,4527625456,Помещение в центре в новом доме,Коммерческая недвижимость,Rielt Group Agency,,21000000,420000,1731690152000,"Санкт-Петербург, Смольный пр-т, 17",...,21–30 мин.,"1,9 км",15 ноября 20:02,Арт. 73114992 Продажа помещения по адресу Смол...,Санкт-Петербург,779.0,59.944292,30.39651,sale,15.11.2024


In [11]:
path_sales = r'O:\Отдел стратегического развития\Данные\Avito\commerce\Sales.xlsx'
path_supply = r'O:\Отдел стратегического развития\Данные\Avito\commerce\Supply.xlsx'

In [12]:
key = 'url'
update_data(new_data, path_sales, path_supply, key)

C:\Users\a.nematov\AppData\Local\Temp\ipykernel_10924\3302081486.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sold_supply.loc[:, 'sale_date'] = date


(1641, 1469)